## SENTIMENT ANALYSIS

In [ ]:
import pandas as pd
import numpy as np
import csv

In [ ]:
covid_ph_df = pd.read_csv("../datasets/covid_philippines/filtered_comments.csv").drop("Unnamed: 0", axis=1)
covid_ph_df.head()

In [ ]:
covid_vax_df = pd.read_csv("../datasets/covid_vaccine/filtered_comments.csv").drop("Unnamed: 0", axis=1)
covid_vax_df.head()

In [ ]:
is_pal_df = pd.read_csv("../datasets/israel-palestine_conflict_history/filtered_comments.csv").drop("Unnamed: 0", axis=1)
is_pal_df.head()

TEST RoBERTa

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import re
from scipy.special import softmax

In [ ]:

'''def roberta_sentiment_scores(df):
    df = df[['video_id','comment']]
    roberta = "cardiffnlp/twitter-roberta-base-sentiment"
    labels = ['Negative', 'Neutral', 'Positive']

    model = AutoModelForSequenceClassification.from_pretrained(roberta)
    tokenizer = AutoTokenizer.from_pretrained(roberta)

    for comments in df['comment']:
        comment_words = []
        comments = comments.replace("\n", " ")
        comments = comments.replace("\xa0", " ")
        comments = comments.replace("?", " ")
        comments = comments.replace(":", " ")
        comments = comments.replace(";", " ")
        comments = comments.replace(";", " ")
        comments = re.sub(r"\s+", ' ', comments) 
        print(comments)
        for word in comments.split(' '):
            if word.startswith('@') and len(word) > 1:
               word = '@user'
        
            elif word.startswith('http'):
                word = "http"
            comment_words.append(word)

        comment_procs = " ".join(comment_words)

        encoded = tokenizer(comment_procs, return_tensors='pt', max_length=512, truncation=True, padding=True)
        #print(encoded)
        output = model(**encoded)

        scores = output[0][0].detach().numpy()

        scores = softmax(scores)

        for i in range(len(scores)):

            l = labels[i]
            s = scores[i]
            df.loc[df['comment']==comments,l] =s
            print(l, s)
    return df'''

In [ ]:
#roberta_df= roberta_sentiment_scores(covid_vax_df)

In [ ]:
#roberta_df

TEST TEXTBLOB

In [ ]:
from textblob import TextBlob


In [ ]:

def textblob_sentiment_scores(input_df):
    df = input_df[['video_id','comment']].copy()
    for comment in df['comment']:
        testimonial = TextBlob(comment)
        df.loc[df['comment']==comment, 'polarity'] =testimonial.sentiment.polarity
        #df.loc[df['comment']==comment, 'subjectivity'] =testimonial.sentiment.subjectivity
        if (testimonial.sentiment.polarity > 0):
            #print("positive", testimonial.sentiment.polarity)
            df.loc[df['comment']==comment, 'sentiment'] ='positive'

        elif (testimonial.sentiment.polarity < 0):
            #print("negative", testimonial.sentiment.polarity)
            df.loc[df['comment']==comment, 'sentiment'] ='negative'
            
        else:
            #print("neutral", testimonial.sentiment.polarity)
            df.loc[df['comment']==comment, 'sentiment'] ='neutral'
    return df

In [233]:
comments_polarity_covid_vax = textblob_sentiment_scores(covid_vax_df)
comments_polarity_covid_ph = textblob_sentiment_scores(covid_ph_df)
comments_polarity_is_pal = textblob_sentiment_scores(is_pal_df)

In [235]:
comments_polarity_covid_vax.to_csv('../datasets/covid_vaccine/per_comment_polarity.csv')
comments_polarity_covid_ph.to_csv('../datasets/covid_philippines/per_comment_polarity.csv')
comments_polarity_is_pal.to_csv('../datasets/israel-palestine_conflict_history/per_comment_polarity.csv')

In [237]:
comments_polarity_is_pal

,video_id,comment,polarity,sentiment
0,R0ftmf_Uv9A,No matter how many times these information get...,-0.166667,negative
1,R0ftmf_Uv9A,Let peace prevail.,0.000000,neutral
2,R0ftmf_Uv9A,Why start at 1946?,0.000000,neutral
3,R0ftmf_Uv9A,What happened the video was removed and put ag...,0.000000,neutral
4,R0ftmf_Uv9A,"This is not a conflict, this is occupation.",0.000000,neutral
...,...,...,...,...
1560,nDtZGbNNbjo,"Salam, Shlomo, Shalom, Peace.\r\n\r\nTakes Two...",0.000000,neutral
1561,nDtZGbNNbjo,Bring peace to Gaza ❤\r\nPALESTINE ❤❤,0.000000,neutral
1562,nDtZGbNNbjo,And the second journalist was standing out in ...,-0.050000,negative
1563,nDtZGbNNbjo,"The crucible purifies silver, and the furnace ...",0.000000,neutral


TEST STANZA

In [ ]:
#import stanza

In [ ]:
'''def stanza_sentiment_scores(input_df):
    df= input_df[['video_id','comment']].copy()
    nlp = stanza.Pipeline('en', processors='tokenize,sentiment', tokenize_no_ssplit=True)

    for comment in df['comment']:
        doc = nlp(comment.replace("\n", " "))
       # print(comment)
    #doc.sentences[0].print_dependencies()
        for i, sentence in enumerate(doc.sentences):
            #The existing models each support negative, neutral, and positive, represented by 0, 1, 2 respectively
            if (sentence.sentiment ==0 ):
                df.loc[df['comment']==comment,'sentiment'] = 'negative'
            elif (sentence.sentiment ==1):
                df.loc[df['comment']==comment,'sentiment'] = 'neutral'
            else:
                df.loc[df['comment']==comment,'sentiment'] = 'positive'

                
    return df
                
            #print("%d -> %d" % (i, sentence.sentiment))'''

In [ ]:
#stanza_sentiment_scores(covid_vax_df)

TEST VADER

In [ ]:
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [ ]:

'''def vader_sentiment_scores (input_df):
    df= input_df[['video_id','comment']].copy()

    for sentence in df['comment']:
        sid_obj = SentimentIntensityAnalyzer()
        sentiment_dict = sid_obj.polarity_scores(sentence)
        #print(sentence)
        #print("Overall sentiment dictionary is : ", sentiment_dict)
        #print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative")
        #print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral")
        #print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive")

        df.loc[df['comment']==sentence,'positive'] = sentiment_dict['pos']
        df.loc[df['comment']==sentence,'neutral'] = sentiment_dict['neu']
        df.loc[df['comment']==sentence,'negative'] = sentiment_dict['neg']

        df.loc[df['comment']==sentence,'compound'] = sentiment_dict['compound']



        #print("Sentence Overall Rated As", end = " ")
 
        # decide sentiment as positive, negative and neutral
        ### compound -> (computed by normalizing the scores above
        if sentiment_dict['compound'] >= 0.05 :
            #print("Positive")
            df.loc[df['comment']==sentence,'Overall sentiment'] = 'positive'

 
        elif sentiment_dict['compound'] <= - 0.05 :
            #print("Negative")
            df.loc[df['comment']==sentence,'Overall sentiment'] = 'negative'
 
        else :
            df.loc[df['comment']==sentence,'Overall sentiment'] = 'neutral'


    return df'''

In [ ]:
#vader_sentiment_scores (covid_vax_df)

## Per Video Comment Sentiment

### TEXTBLOB APPROACH

In [249]:
def per_vid_textblob_polarity (df_input):
    df = pd.DataFrame(df_input["video_id"].copy())
    df =df.drop_duplicates('video_id')
    df= df.reset_index(drop=True)
    for video in df_input["video_id"].unique():
        sum_polarity=0
        for comment in df_input.loc[df_input["video_id"]==video]["comment"]:
            sum_polarity+=(TextBlob(comment)).sentiment.polarity
        average=sum_polarity/df_input.loc[df_input["video_id"]==video]["comment"].count()
        df.loc[df["video_id"]==video,'polarity'] =average
       # video_polarity[video] = [sum_polarity/df.loc[df["video_id"]==video]["comment"].count()]
        if (average > 0):
            df.loc[df["video_id"]==video,'sentiment'] ='positive'
        elif (average < 0):
            df.loc[df["video_id"]==video,'sentiment'] ='negative'
        else:
            df.loc[df["video_id"]==video,'sentiment'] ='neutral'
    
    return df

In [250]:
comments_per_vid_vax = per_vid_textblob_polarity (covid_vax_df)
comments_per_vid_ph = per_vid_textblob_polarity (covid_ph_df)
comments_per_vid_pal = per_vid_textblob_polarity(is_pal_df)

In [252]:
comments_per_vid_vax.to_csv('../datasets/covid_vaccine/per_video_comments_polarity.csv')
comments_per_vid_ph.to_csv('../datasets/covid_philippines/per_video_comment_polarity.csv')
comments_per_vid_pal.to_csv('../datasets/israel-palestine_conflict_history/per_video_comment_polarity.csv')

In [243]:
comments_per_vid_ph

,video_id,polarity,sentiment
0,aLZ85hb4wjE,0.222917,positive
1,sdsz-t540WI,0.063264,positive
2,sYI97jv-pZg,-0.300000,negative
3,pMUumjHY3tw,-0.050833,negative
4,DWxIvQlpJK8,0.006944,positive
...,...,...,...
163,L0RRohlPzOc,0.196492,positive
164,9ehc2TyH2ak,0.142003,positive
165,MWmIqdwy4Ww,0.042857,positive
166,lqbQhDHBuNQ,-0.012970,negative


In [247]:
comments_per_vid_vax

,video_id,polarity,sentiment
0,2IXl4qJGrRk,0.202778,positive
1,HtTalpY-J-M,0.050000,positive
2,jPs4_MeuX7U,0.095748,positive
3,WhiBpmH1mE4,-0.004616,negative
4,LfmhYVCCGhc,0.167498,positive
...,...,...,...
163,l6mTAJ08aUI,-0.016204,negative
164,2XVCPdPAUbA,0.030952,positive
165,XHfRpNJI0c8,0.088384,positive
166,wuqyRBmashA,-0.015464,negative


### VADER APPROACH

In [ ]:
'''def per_vid_vader_polarity (df_input):
    df = pd.DataFrame(df_input["video_id"].copy())
    df =df.drop_duplicates('video_id')
    df= df.reset_index(drop=True)
    polarity_scores={}
    sid_obj = SentimentIntensityAnalyzer()

    for video in df_input["video_id"].unique():
        sum_neg =0
        sum_pos = 0
        sum_neu =0
        sum_compound= 0
        scores={}
        for sentence in df_input.loc[df_input["video_id"]==video]["comment"]:

            sentiment_dict = sid_obj.polarity_scores(sentence)
            sum_neg += sentiment_dict['neg']
            sum_pos += sentiment_dict['pos']
            sum_neu += sentiment_dict['neu']
            sum_compound += sentiment_dict['compound']
            
        df.loc[df['video_id']==video,'negative'] = sum_neg/df_input.loc[df_input["video_id"]==video]["comment"].count()
        df.loc[df['video_id']==video,'positive'] = sum_pos/df_input.loc[df_input["video_id"]==video]["comment"].count()
        df.loc[df['video_id']==video,'neutral'] = sum_neu/df_input.loc[df_input["video_id"]==video]["comment"].count()
        df.loc[df['video_id']==video,'compound'] = sum_compound/df_input.loc[df_input["video_id"]==video]["comment"].count()
        
        #polarity_scores[video]=scores
        # decide sentiment as positive, negative and neutral
        if sentiment_dict['compound'] >= 0.05 :
            print("Positive")
 
        elif sentiment_dict['compound'] <= - 0.05 :
            print("Negative")
 
        else :
            print("Neutral")
    return df'''
        

In [ ]:
#per_vid_vader_polarity(covid_vax_df)

### RoBERTa APPROACH

In [ ]:
#from transformers import AutoTokenizer, AutoModelForSequenceClassification
#import re
#from scipy.special import softmax

'''def roberta_per_vid_scores(df_input):
    df = pd.DataFrame(df_input["video_id"].copy())
    df =df.drop_duplicates('video_id')
    df= df.reset_index(drop=True)
    polarity_scores={}

    roberta = "cardiffnlp/twitter-roberta-base-sentiment"
    labels = ['Negative', 'Neutral', 'Positive']

    model = AutoModelForSequenceClassification.from_pretrained(roberta)
    tokenizer = AutoTokenizer.from_pretrained(roberta)
    for video in df_input["video_id"].unique():
        sum_neg =0
        sum_pos = 0
        sum_neu =0 
        scores={}
        for comments in df_input.loc[df_input["video_id"]==video]["comment"]:
            comment_words = []
            comments = comments.replace("\n", " ")
            comments = comments.replace("\xa0", " ")
            comments = comments.replace("?", " ")
            comments = comments.replace(":", " ")
            comments = comments.replace(";", " ")
            comments = comments.replace(";", " ")
            comments = re.sub(r"\s+", ' ', comments) 
     #   print(comments)
            for word in comments.split(' '):
                if word.startswith('@') and len(word) > 1:
                    word = '@user'
        
                elif word.startswith('http'):
                    word = "http"
                comment_words.append(word)

            comment_procs = " ".join(comment_words)

            encoded = tokenizer(comment_procs, return_tensors='pt', max_length=512, truncation=True, padding=True)
           # print(encoded)
            output = model(**encoded)

            scores = output[0][0].detach().numpy()

            scores = softmax(scores)

            for i in range(len(scores)):

                l = labels[i]
                s = scores[i]
                #    labels = ['Negative', 'Neutral', 'Positive']

                if l=="Negative":
                    sum_neg+= s
                elif l == "Neutral":
                    sum_neu+=s
                else : 
                    sum_pos+=s
        
        df.loc[df['video_id']==video,'negative'] = sum_neg/df_input.loc[df_input["video_id"]==video]["comment"].count()
        df.loc[df['video_id']==video,'positive'] = sum_pos/df_input.loc[df_input["video_id"]==video]["comment"].count()
        df.loc[df['video_id']==video,'neutral'] = sum_neu/df_input.loc[df_input["video_id"]==video]["comment"].count()


       # scores["Neg"] = sum_neg/df.loc[df["video_id"]==video]["comment"].count()
        #scores =dict([("Neg", sum_neg/df.loc[df["video_id"]==video]["comment"].count()),("Pos",sum_pos/df.loc[df["video_id"]==video]["comment"].count()),("Neu",sum_neu/df.loc[df["video_id"]==video]["comment"].count()
#)])
        
        #scores["Pos"] = sum_pos/df.loc[df["video_id"]==video]["comment"].count()
        #scores["Neu"] = sum_neu/df.loc[df["video_id"]==video]["comment"].count()

        #polarity_scores[video] = scores
    return df'''

In [ ]:
#roberta_per_vid_scores(covid_vax_df)

### STANZA

In [ ]:
'''def stanza_pervid_sentiment(input_df):
    df= input_df[['video_id','comment']].copy()
    video_df = pd.DataFrame(input_df["video_id"].copy())
    video_df =video_df.drop_duplicates('video_id')
    video_df= video_df.reset_index(drop=True)

    nlp = stanza.Pipeline('en', processors='tokenize,sentiment', tokenize_no_ssplit=True)

    for comment in df['comment']:
        doc = nlp(comment.replace("\n", " "))
       # print(comment)
    #doc.sentences[0].print_dependencies()
        for i, sentence in enumerate(doc.sentences):
            #The existing models each support negative, neutral, and positive, represented by 0, 1, 2 respectively
            df.loc[df['comment']==comment,'sentiment'] = sentence.sentiment
            
    for video in video_df['video_id']:            
        video_df.loc[video_df['video_id']==video,'mode_sentiment'] = df.mode()['sentiment'][0]
                
    return video_df
                
            #print("%d -> %d" % (i, sentence.sentiment))'''

In [ ]:
#stanza_pervid_sentiment(covid_vax_df)
